In [1]:
%cd '/content/drive/MyDrive/UCO/projects/epilepsy'

/content/drive/MyDrive/UCO/projects/epilepsy


In [2]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.7 MB/s eta 0:00:00


In [3]:
# prompt: import data_prep.py in the directory

import sys
sys.path.append('./') # Assuming data_prep.py is in the same directory
print(sys.path)
from model import HeteroGAT3
from data_prep import heterdataPrep
from util import balance_homodata_underSample, load_homodata, train_test_split_homodata
from test_sample_size import test_sample_size

['/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/usr/local/lib/python3.11/dist-packages/setuptools/_vendor', '/root/.ipython', './']
['/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/usr/local/lib/python3.11/dist-packages/setuptools/_vendor', '/root/.ipython', './', '/tmp/tmp1b4n7swu', './']


In [4]:
from torch_geometric.utils import subgraph
import torch
import h5py
import math
from torch_geometric.data import HeteroData

from torch.utils.data import Dataset


def train_test_split_homodata(data, train_ratio=0.8, offset = 20):

  # Generate a random permutation of indices
  random_indices = torch.randperm(len(data.y)-1)
  train_ratio = 0.8
  # Calculate the number of training samples
  num_train = int(train_ratio *( len(data.y)-1))

  # Create the train mask based on the random permutation
  train_mask = torch.zeros(len(data.y)-1, dtype=torch.bool)
  train_mask[random_indices[:num_train]] = True

  # Create the test mask (complement of the train mask)
  test_mask = ~train_mask



  # Get indices of training and testing nodes
  train_indices = torch.where(train_mask)[0]
  test_indices = torch.where(test_mask)[0]


  # print(test_indices)
  # print(len(train_indices))
#   print(torch.range(0,offset))
  # print( len(torch.cat([torch.range(0,offset-1),train_indices + offset], dim=0).int()))




  # Use subgraph to extract training and testing data
  train_data = data.clone()
  train_data.y = data.y[train_indices]
  train_data.x = data.x[torch.cat([torch.tensor(range(0,offset)),train_indices + offset], dim=0).int()].float()
  train_data.edge_index, train_data.edge_attr = subgraph(
     torch.cat([torch.tensor(range(0,offset)),train_indices + offset], dim=0).int(), data.edge_index, data.edge_attr, relabel_nodes=True)


  test_data = data.clone()
  test_data.edge_index, test_data.edge_attr = subgraph(
      torch.cat([torch.tensor(range(0,offset)),test_indices + offset], dim=0).int(), data.edge_index, data.edge_attr, relabel_nodes=True)
  test_data.y = data.y[test_indices]
  test_data.x = data.x[torch.cat([torch.tensor(range(0,offset)),test_indices + offset], dim=0).int()].float()

  return train_data, test_data




In [5]:
import sys
import torch
from torch.utils.data import DataLoader
sys.path.append('./')  # Assuming data_prep.py is in the same directory
print(sys.path)
from model import HeteroGAT3, HeteroGAT4
from data_prep import heterdataPrep, connect_diagnosis_edges
from util import balance_homodata_underSample, load_homodata,  CustomGraphDataset
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve, auc, roc_auc_score
from tqdm import tqdm


def predict_by_window(file, pred_win, offset, batch_size, hidden_channels, convhidden_channels, num_layers, num_heads, lr, n_epoch, self_connect_diag=False):
    data = load_homodata(file)
    # Check if GPU is available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    print(data.edge_attr)
    data = filter_graph_len_como(data, pred_win)
    print(data.edge_attr)


    train_data, test_data = train_test_split_homodata(data, 0.8, offset)
    train_data = balance_homodata_underSample(train_data, offset)
    train_data_homo = train_data.clone()

    test_data_balanced = balance_homodata_underSample(test_data, offset)
    train_data = heterdataPrep(train_data, offset, agg=False)
    test_data = heterdataPrep(test_data, offset, agg=False)
    test_data_balanced = heterdataPrep(test_data_balanced, offset, agg=False)

    train_data_d2d = connect_diagnosis_edges(train_data)

    test_data_d2d = connect_diagnosis_edges(test_data)

    test_data_balanced_d2d = connect_diagnosis_edges(test_data_balanced)

    if self_connect_diag:
        GAT2 = HeteroGAT4(input_features=3, edge_features=1, hidden_channels=hidden_channels,
                          convhidden_channels=convhidden_channels, out_channels=2, num_heads=num_heads,
                          metadata=train_data.metadata(), num_layers=num_layers).to(device)
    else:
        GAT2 = HeteroGAT3(input_features=3, edge_features=1, hidden_channels=hidden_channels,
                          convhidden_channels=convhidden_channels, out_channels=2, num_heads=num_heads,
                          metadata=train_data.metadata(), num_layers=num_layers).to(device)

    optimizer = torch.optim.Adam(GAT2.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()

    if batch_size == 'full':
        if self_connect_diag:
            train_data = train_data_d2d
            test_data = test_data_d2d
            test_data_balanced = test_data_balanced_d2d

        train_data, test_data, test_data_balanced = train_data.to(device), test_data.to(device), test_data_balanced.to(device)

        progress_bar = tqdm(range(n_epoch))
        for epoch in progress_bar:
            GAT2.train()
            optimizer.zero_grad()
            out = GAT2(train_data.x_dict, train_data.edge_index_dict, train_data.edge_attr_dict)
            loss = criterion(out, train_data['person'].y)
            loss.backward()
            optimizer.step()
            progress_bar.set_postfix({'loss': loss.item()})

        print('train performance:')
        eval_mode(GAT2, train_data)
        print('test performance:')
        eval_mode(GAT2, test_data)
        # print('test_balanced performance')
        # eval_mode(GAT2, test_data_balanced)

    else:
        # Use CustomGraphDataset for subgraph generation
        train_dataset = CustomGraphDataset(train_data_homo, batch_size, offset)
        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=hetero_collate_fn)


        if self_connect_diag:
            train_data = train_data_d2d
            test_data = test_data_d2d

        train_data, test_data, test_data_balanced = train_data.to(device), test_data.to(device), test_data_balanced.to(device)

        for epoch in range(n_epoch):
            progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{n_epoch}")
            avg_loss = 0.0
            for batch in progress_bar:
                batch = batch[0]
                batch = heterdataPrep(batch, offset, agg=False)
                if self_connect_diag:
                  batch = connect_diagnosis_edges(batch)
                # print(batch)
                subgraph = batch.to(device)  # Extract and send the subgraph to the device
                GAT2.train()
                optimizer.zero_grad()
                out = GAT2(subgraph.x_dict, subgraph.edge_index_dict, subgraph.edge_attr_dict)
                loss = criterion(out, subgraph['person'].y)
                loss.backward()
                optimizer.step()
                progress_bar.set_postfix({'loss': loss.item()})
                avg_loss = avg_loss + loss.item()

            print('avg_loss',avg_loss/len(train_dataloader))

            if epoch % 2 == 0: # print evaluation report every 2 epochs
                print('train performance:')
                eval_mode(GAT2, train_data)
                print('test performance:')
                eval_mode(GAT2, test_data)
                # print('test_balanced performance')
                # eval_mode(GAT2, test_data_balanced)

    return GAT2

def eval_mode(GAT2, data_test):
    # Evaluate the model
    GAT2.eval()
    # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # print(f"Using device: {device}")

    # data_test = data_test.to(device)
    with torch.no_grad():
        out = GAT2(data_test.x_dict, data_test.edge_index_dict, data_test.edge_attr_dict)
        _, predicted = torch.max(out, dim=1)  # Predictions for persons only

    # Get the confusion matrix
    cm = confusion_matrix(data_test['person'].y.cpu(), predicted.cpu())
    print("Confusion Matrix:")
    print(cm)

    # Extract True Positives, False Positives, False Negatives, and True Negatives
    tn, fp, fn, tp = cm.ravel()

    # Calculate sensitivity and specificity
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0

    print(f"Sensitivity (Recall): {sensitivity:.4f}")
    print(f"Specificity: {specificity:.4f}")

    # Print classification report
    print("Classification Report:")
    print(classification_report(data_test['person'].y.cpu(), predicted.cpu()))

    # Extract probabilities for the positive class (class 1)
    probabilities = torch.softmax(out, dim=1)[:, 1].cpu().numpy()

    # Calculate the AUC-ROC
    aucroc = roc_auc_score(data_test['person'].y.cpu(), probabilities)
    print(f"AUC-ROC: {aucroc:.4f}")

    # Calculate precision-recall curve
    precision, recall, thresholds = precision_recall_curve(data_test['person'].y.cpu(), probabilities)

    # Calculate PRAUC
    prauc = auc(recall, precision)
    print(f"PR-AUC: {prauc:.4f}")


def filter_graph_len_como(homodata, pred_win):
   filtered = homodata.clone()
   mask = homodata.edge_attr[:,0] >= pred_win  # Create a mask for edges that meet the threshold condition
   filtered.edge_attr = homodata.edge_attr[mask]  # Apply mask to edge attributes
   filtered.edge_index = homodata.edge_index[:, mask]  # Apply mask to edge index

   return filtered




['/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/usr/local/lib/python3.11/dist-packages/setuptools/_vendor', '/root/.ipython', './', '/tmp/tmp1b4n7swu', './', './']


In [ ]:
GAT500=predict_by_window('/content/drive/MyDrive/UCO/projects/epilepsy/EPI_heterodata_diag500_first.h5',pred_win=1/12, offset = 500, batch_size = 'full', hidden_channels = 32, convhidden_channels = 32, num_heads = 4, num_layers = 2, n_epoch = 1000, lr = 0.001, self_connect_diag = True)

Using device: cuda
tensor([[0.1760],
        [0.1760],
        [0.1769],
        ...,
        [0.0000],
        [0.0000],
        [0.0000]])
tensor([[0.1760],
        [0.1760],
        [0.1769],
        ...,
        [0.5484],
        [0.5341],
        [0.5341]])
class counts =  tensor([1279352,   81995])
minority_class:  tensor(1)
majority_class:  tensor(0)
class counts =  tensor([319645,  20692])
minority_class:  tensor(1)
majority_class:  tensor(0)


100%|██████████| 800/800 [06:47<00:00,  1.96it/s, loss=0.604]


train performance:
Confusion Matrix:
[[37881 44114]
 [13157 68838]]
Sensitivity (Recall): 0.8395
Specificity: 0.4620
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.46      0.57     81995
           1       0.61      0.84      0.71     81995

    accuracy                           0.65    163990
   macro avg       0.68      0.65      0.64    163990
weighted avg       0.68      0.65      0.64    163990

AUC-ROC: 0.6954
PR-AUC: 0.7399
test performance:
Confusion Matrix:
[[139840 179805]
 [  4160  16532]]
Sensitivity (Recall): 0.7990
Specificity: 0.4375
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.44      0.60    319645
           1       0.08      0.80      0.15     20692

    accuracy                           0.46    340337
   macro avg       0.53      0.62      0.38    340337
weighted avg       0.92      0.46      0.58    340337

AUC-ROC: 0.6492
PR-AUC: 0.1246


In [7]:
GAT500=predict_by_window('/content/drive/MyDrive/UCO/projects/epilepsy/EPI_heterodata_diag500_first.h5',pred_win=1/12, offset = 500, batch_size = 'full', hidden_channels = 32, convhidden_channels = 32, num_heads = 4, num_layers = 2, n_epoch = 1000, lr = 0.001, self_connect_diag = False)

Using device: cuda
tensor([[0.1760],
        [0.1760],
        [0.1769],
        ...,
        [0.0000],
        [0.0000],
        [0.0000]])
tensor([[0.1760],
        [0.1760],
        [0.1769],
        ...,
        [0.5484],
        [0.5341],
        [0.5341]])
class counts =  tensor([1279217,   82130])
minority_class:  tensor(1)
majority_class:  tensor(0)
class counts =  tensor([319780,  20557])
minority_class:  tensor(1)
majority_class:  tensor(0)


100%|██████████| 1000/1000 [08:24<00:00,  1.98it/s, loss=0.601]


train performance:
Confusion Matrix:
[[39180 42950]
 [14224 67906]]
Sensitivity (Recall): 0.8268
Specificity: 0.4770
Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.48      0.58     82130
           1       0.61      0.83      0.70     82130

    accuracy                           0.65    164260
   macro avg       0.67      0.65      0.64    164260
weighted avg       0.67      0.65      0.64    164260

AUC-ROC: 0.6977
PR-AUC: 0.7510
test performance:
Confusion Matrix:
[[151772 168008]
 [  4890  15667]]
Sensitivity (Recall): 0.7621
Specificity: 0.4746
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.47      0.64    319780
           1       0.09      0.76      0.15     20557

    accuracy                           0.49    340337
   macro avg       0.53      0.62      0.40    340337
weighted avg       0.92      0.49      0.61    340337

AUC-ROC: 0.6473
PR-AUC: 0.1252


In [ ]:
GAT500=predict_by_window('/content/drive/MyDrive/UCO/projects/epilepsy/EPI_heterodata_diag500_first.h5',pred_win=0.5, offset = 500, batch_size = 'full', hidden_channels = 32, convhidden_channels = 32, num_heads = 4, num_layers = 2, n_epoch = 800, lr = 0.0001, self_connect_diag = True)

Using device: cuda
tensor([[0.1760],
        [0.1760],
        [0.1769],
        ...,
        [0.0000],
        [0.0000],
        [0.0000]])
class counts =  tensor([1279248,   82099])
minority_class:  tensor(1)
majority_class:  tensor(0)
class counts =  tensor([319749,  20588])
minority_class:  tensor(1)
majority_class:  tensor(0)


100%|██████████| 800/800 [05:37<00:00,  2.37it/s, loss=0.662]


train performance:
Confusion Matrix:
[[32235 49864]
 [17249 64850]]
Sensitivity (Recall): 0.7899
Specificity: 0.3926
Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.39      0.49     82099
           1       0.57      0.79      0.66     82099

    accuracy                           0.59    164198
   macro avg       0.61      0.59      0.57    164198
weighted avg       0.61      0.59      0.57    164198

AUC-ROC: 0.6092
PR-AUC: 0.6472
test performance:
Confusion Matrix:
[[138331 181418]
 [  5367  15221]]
Sensitivity (Recall): 0.7393
Specificity: 0.4326
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.43      0.60    319749
           1       0.08      0.74      0.14     20588

    accuracy                           0.45    340337
   macro avg       0.52      0.59      0.37    340337
weighted avg       0.91      0.45      0.57    340337

AUC-ROC: 0.6063
PR-AUC: 0.0617


In [ ]:
GAT500=predict_by_window('/content/drive/MyDrive/UCO/projects/epilepsy/EPI_heterodata_diag500_first.h5',pred_win=0.25, offset = 500, batch_size = 'full', hidden_channels = 32, convhidden_channels = 32, num_heads = 4, num_layers = 2, n_epoch = 800, lr = 0.0001, self_connect_diag = True)

Using device: cuda
tensor([[0.1760],
        [0.1760],
        [0.1769],
        ...,
        [0.0000],
        [0.0000],
        [0.0000]])
tensor([[7.0689],
        [7.0687],
        [7.0687],
        ...,
        [0.5484],
        [0.5341],
        [0.5341]])
class counts =  tensor([1279111,   82236])
minority_class:  tensor(1)
majority_class:  tensor(0)
class counts =  tensor([319886,  20451])
minority_class:  tensor(1)
majority_class:  tensor(0)


100%|██████████| 800/800 [06:21<00:00,  2.10it/s, loss=0.618]


train performance:
Confusion Matrix:
[[34751 47485]
 [12005 70231]]
Sensitivity (Recall): 0.8540
Specificity: 0.4226
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.42      0.54     82236
           1       0.60      0.85      0.70     82236

    accuracy                           0.64    164472
   macro avg       0.67      0.64      0.62    164472
weighted avg       0.67      0.64      0.62    164472

AUC-ROC: 0.6766
PR-AUC: 0.7119
test performance:
Confusion Matrix:
[[112014 207872]
 [  2995  17456]]
Sensitivity (Recall): 0.8536
Specificity: 0.3502
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.35      0.52    319886
           1       0.08      0.85      0.14     20451

    accuracy                           0.38    340337
   macro avg       0.53      0.60      0.33    340337
weighted avg       0.92      0.38      0.49    340337

AUC-ROC: 0.6127
PR-AUC: 0.1061


In [ ]:
GAT500=predict_by_window('/content/drive/MyDrive/UCO/projects/epilepsy/EPI_heterodata_diag500_first.h5',pred_win=0, offset = 500, batch_size = 'full', hidden_channels = 32, convhidden_channels = 32, num_heads = 4, num_layers = 2, n_epoch = 800, lr = 0.0001, self_connect_diag = True)

Using device: cuda
class counts =  tensor([1279388,   81959])
minority_class:  tensor(1)
majority_class:  tensor(0)
class counts =  tensor([319609,  20728])
minority_class:  tensor(1)
majority_class:  tensor(0)


100%|██████████| 800/800 [08:16<00:00,  1.61it/s, loss=0.523]


train performance:
Confusion Matrix:
[[53128 28831]
 [12405 69554]]
Sensitivity (Recall): 0.8486
Specificity: 0.6482
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.65      0.72     81959
           1       0.71      0.85      0.77     81959

    accuracy                           0.75    163918
   macro avg       0.76      0.75      0.75    163918
weighted avg       0.76      0.75      0.75    163918

AUC-ROC: 0.8028
PR-AUC: 0.7864
test performance:
Confusion Matrix:
[[218043 101566]
 [  4248  16480]]
Sensitivity (Recall): 0.7951
Specificity: 0.6822
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.68      0.80    319609
           1       0.14      0.80      0.24     20728

    accuracy                           0.69    340337
   macro avg       0.56      0.74      0.52    340337
weighted avg       0.93      0.69      0.77    340337

AUC-ROC: 0.7861
PR-AUC: 0.1979


In [ ]:
GAT500=predict_by_window('/content/drive/MyDrive/UCO/projects/epilepsy/EPI_heterodata_diag500_first.h5',pred_win=1, offset = 500, batch_size = 'full', hidden_channels = 32, convhidden_channels = 32, num_heads = 4, num_layers = 2, n_epoch = 800, lr = 0.0001, self_connect_diag = True)

Using device: cuda
class counts =  tensor([1279267,   82080])
minority_class:  tensor(1)
majority_class:  tensor(0)
class counts =  tensor([319730,  20607])
minority_class:  tensor(1)
majority_class:  tensor(0)


100%|██████████| 800/800 [04:44<00:00,  2.81it/s, loss=0.657]


train performance:
Confusion Matrix:
[[30526 51554]
 [15220 66860]]
Sensitivity (Recall): 0.8146
Specificity: 0.3719
Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.37      0.48     82080
           1       0.56      0.81      0.67     82080

    accuracy                           0.59    164160
   macro avg       0.62      0.59      0.57    164160
weighted avg       0.62      0.59      0.57    164160

AUC-ROC: 0.6147
PR-AUC: 0.6909
test performance:
Confusion Matrix:
[[120448 199282]
 [  4096  16511]]
Sensitivity (Recall): 0.8012
Specificity: 0.3767
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.38      0.54    319730
           1       0.08      0.80      0.14     20607

    accuracy                           0.40    340337
   macro avg       0.52      0.59      0.34    340337
weighted avg       0.91      0.40      0.52    340337

AUC-ROC: 0.6079
PR-AUC: 0.1344


In [ ]:
GAT500=predict_by_window('/content/drive/MyDrive/UCO/projects/epilepsy/EPI_heterodata_diag500_first.h5',pred_win=2, offset = 500, batch_size = 'full', hidden_channels = 32, convhidden_channels = 32, num_heads = 4, num_layers = 2, n_epoch = 800, lr = 0.0001, self_connect_diag = True)

Using device: cuda
tensor([0.1760, 0.1760, 0.1769,  ..., 0.0000, 0.0000, 0.0000])
tensor([False, False, False,  ..., False, False, False])
tensor([[1693956, 1693956, 1693956,  ...,  139021,  139021,  139021],
        [    258,     471,     471,  ...,      42,     112,     298]])
class counts =  tensor([1279075,   82272])
minority_class:  tensor(1)
majority_class:  tensor(0)
class counts =  tensor([319922,  20415])
minority_class:  tensor(1)
majority_class:  tensor(0)


100%|██████████| 800/800 [03:23<00:00,  3.92it/s, loss=0.663]


train performance:
Confusion Matrix:
[[21227 61045]
 [ 8315 73957]]
Sensitivity (Recall): 0.8989
Specificity: 0.2580
Classification Report:
              precision    recall  f1-score   support

           0       0.72      0.26      0.38     82272
           1       0.55      0.90      0.68     82272

    accuracy                           0.58    164544
   macro avg       0.63      0.58      0.53    164544
weighted avg       0.63      0.58      0.53    164544

AUC-ROC: 0.5944
PR-AUC: 0.6058
test performance:
Confusion Matrix:
[[ 96436 223486]
 [  3622  16793]]
Sensitivity (Recall): 0.8226
Specificity: 0.3014
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.30      0.46    319922
           1       0.07      0.82      0.13     20415

    accuracy                           0.33    340337
   macro avg       0.52      0.56      0.29    340337
weighted avg       0.91      0.33      0.44    340337

AUC-ROC: 0.5710
PR-AUC: 0.0678
